In [94]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [95]:
df = pd.read_csv("heart_failure_clinical_records_dataset.csv")
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [96]:
df.shape

(299, 13)

In [97]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [98]:
df.isnull().values.any()   #checking for null values currently our dataset does not have any null values

False

In [99]:
## If there were any null values we can replace the null values with either mean or median of the coloumn
## For instance if "age" coloumn is having a null value we can follow the procedure as,
# m = df['Age'].mean()
# df['age'].fillna(value=m,inplace=True)

In [100]:
## smote for balancing dataset
from imblearn.over_sampling import SMOTE
target = df["DEATH_EVENT"]
sm  = SMOTE(random_state=12)
X_res, y_res= sm.fit_resample(df.drop(df[["DEATH_EVENT"]],axis="columns"),target)
smoted_df = pd.concat([X_res,y_res], axis=1)
print(len(smoted_df))


406


In [101]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_res,y_res,test_size=0.2)
count,unique = np.unique(y_res,return_counts=True)
print(count,unique)

[0 1] [203 203]


In [102]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train,y_train)
y_res_pred = logistic_model.predict(X_test)
logistic_model.score(X_test,y_test)

0.7682926829268293

In [103]:
## removing outliers using quantile 

max_thresold_age = smoted_df["age"].quantile(0.90)
min_thresold_age = smoted_df["age"].quantile(0.01)
print("max age:",max_thresold_age,"min age:",min_thresold_age)


max age: 78.96805462096657 min age: 40.0


In [104]:
len(smoted_df[smoted_df["age"]>max_thresold_age])

41

In [105]:
len(smoted_df[smoted_df["age"]<min_thresold_age])

0

In [106]:

df1 = smoted_df[(smoted_df.age>min_thresold_age) & (smoted_df.age<max_thresold_age)]
len(df1)

358

In [107]:
from sklearn.linear_model import LogisticRegression
target = df1["DEATH_EVENT"]
X_train,X_test,y_train,y_test = train_test_split(df1.drop(df1[["DEATH_EVENT"]],axis="columns"),target,test_size=0.2)
logistic_model = LogisticRegression()
logistic_model.fit(X_train,y_train)
y_res_pred = logistic_model.predict(X_test)
logistic_model.score(X_test,y_test)

0.7361111111111112

In [109]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
gbc = GradientBoostingClassifier(learning_rate=0.1)
gbc.fit(X_train,y_train)
gbc.score(X_test,y_test)
#print(classification_report(y_test,gbc.predict(X_test)))

0.875

In [150]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=5,criterion='entropy',max_features=8)
etc.fit(X_train,y_train)
etc.score(X_test,y_test)
feature_imp = etc.feature_importances_
feature_imp_norm = np.std([tree.feature_importances_ for tree in etc.estimators_],axis=0)
print(feature_imp*100)

[ 4.21644503  2.92730843  2.95703617  5.48972747 17.98537926  4.01774726
  3.59861903 18.53811442  4.15048599  3.44906924  4.12145729 28.54861041]
